# 🚀 YOLOv11s 소나무 피해목 탐지 - 원클릭 학습 시스템

## 🎯 **4단계 원클릭 실행**

### **⚡ 빠른 시작**
1. **🖥️ GPU 설정**: `런타임` → `런타임 유형 변경` → `T4 GPU` 선택
2. **🚀 원클릭 실행**: `런타임` → `모두 실행` 클릭  
3. **☕ 대기**: 약 1-2시간 후 완성!

### **🔄 자동 실행 단계**
- **1단계**: 환경 설정 및 기존 모델 탐지
- **2단계**: YOLOv11s 소나무 전용 학습
- **3단계**: 결과 분석 및 시각화
- **4단계**: 모델 성능 평가

### **🎯 네이밍 규칙**
- **프로젝트**: `pinetree_yolov11s` (유지)
- **실행명**: `damage_detection_YYYYMMDD_HHMM`
- **예시**: `damage_detection_20250916_1430`

### **? 예상 성능**
- **mAP50**: 93%+ (높은 탐지 정확도)
- **Precision**: 90%+ (오탐지 최소화)  
- **Recall**: 85%+ (실제 피해목 탐지)

---

## 🔧 **1단계: 환경 설정 및 기존 모델 탐지**

> 패키지 설치, 기존 학습된 모델 자동 탐지, GPU 설정을 모두 자동화합니다.

In [ ]:
# 🔧 1단계: 환경 설정 및 기존 모델 탐지
import subprocess
import sys
import os
import glob
from datetime import datetime
from pathlib import Path

def install_package(package):
    """패키지 자동 설치"""
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} 이미 설치됨")
    except ImportError:
        print(f"📦 {package} 설치 중...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 완료")

print("🚀 YOLOv11s 소나무 피해목 탐지 - 환경 설정")
print("=" * 50)

# 🗂️ Google Drive 마운트 및 ZIP 데이터 자동 압축 해제 (Colab 환경에서만)
try:
    import google.colab
    IN_COLAB = True
    print("📍 Google Colab 환경 감지")
    
    # Google Drive 마운트
    print("🔗 Google Drive 마운트 중...")
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 마운트 완료")
    
    # 작업 디렉토리 설정
    drive_path = '/content/drive/MyDrive/pinetree_scan'
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"📁 작업 디렉토리 생성: {drive_path}")
    
    # 🔍 ZIP 파일 자동 탐지 및 압축 해제 함수
    def find_and_extract_training_zip():
        """Google Drive에서 training_data ZIP 파일을 찾아 압축 해제"""
        import zipfile
        import shutil
        
        print("\n🔍 Google Drive에서 ZIP 데이터셋 탐지 중...")
        
        # ZIP 파일 검색 경로 (우선순위별)
        zip_search_paths = [
            # 1순위: pinetree_scan/training_data 폴더 내의 모든 ZIP 파일
            '/content/drive/MyDrive/pinetree_scan/training_data/*.zip',
            
            # 2순위: pinetree_scan 폴더 내
            '/content/drive/MyDrive/pinetree_scan/training_data.zip',
            '/content/drive/MyDrive/pinetree_scan/dataset.zip', 
            '/content/drive/MyDrive/pinetree_scan/data.zip',
            
            # 3순위: 직접 업로드
            '/content/drive/MyDrive/training_data.zip',
            '/content/drive/MyDrive/dataset.zip',
            '/content/drive/MyDrive/data.zip',
            '/content/drive/MyDrive/yolo_data.zip',
            '/content/drive/MyDrive/pine_data.zip',
            
            # 4순위: 와일드카드 검색
            '/content/drive/MyDrive/*training*.zip',
            '/content/drive/MyDrive/*dataset*.zip',
            '/content/drive/MyDrive/*data*.zip',
            '/content/drive/MyDrive/pinetree_scan/*training*.zip',
            '/content/drive/MyDrive/pinetree_scan/*dataset*.zip'
        ]
        
        found_zips = []
        
        for search_path in zip_search_paths:
            if '*' in search_path:
                # 와일드카드 검색
                import glob
                matches = glob.glob(search_path)
                for match in matches:
                    if os.path.exists(match) and match.endswith('.zip'):
                        found_zips.append(match)
            else:
                # 직접 경로 확인
                if os.path.exists(search_path):
                    found_zips.append(search_path)
        
        if found_zips:
            # 중복 제거
            found_zips = list(set(found_zips))
            
            print(f"📦 발견된 ZIP 파일들:")
            for i, zip_file in enumerate(found_zips, 1):
                file_size = os.path.getsize(zip_file) / 1024 / 1024  # MB
                print(f"  {i}. {os.path.basename(zip_file)} ({file_size:.1f}MB)")
                print(f"     경로: {zip_file}")
            
            # 가장 큰 파일을 선택 (훈련 데이터가 클 가능성이 높음)
            zip_path = max(found_zips, key=os.path.getsize)
            print(f"\n✅ 선택된 파일: {os.path.basename(zip_path)}")
            print(f"📁 경로: {zip_path}")
            
            # 파일 크기 확인
            zip_size = os.path.getsize(zip_path) / 1024 / 1024  # MB
            print(f"📏 크기: {zip_size:.1f}MB")
            
            # 압축 해제 대상 경로
            extract_path = '/content/drive/MyDrive/pinetree_scan/training_data'
            temp_extract_path = '/content/training_data_temp'
            
            try:
                print(f"\n🔄 ZIP 파일 압축 해제 중...")
                
                # 임시 경로에 압축 해제
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(temp_extract_path)
                
                print(f"✅ 압축 해제 완료: {temp_extract_path}")
                
                # 압축 해제된 내용 확인 및 정리
                extracted_items = os.listdir(temp_extract_path)
                print(f"📋 압축 해제 내용: {extracted_items}")
                
                # training_data 폴더 구조 정리
                final_path = extract_path
                
                # 기존 training_data가 있으면 백업
                if os.path.exists(final_path):
                    backup_path = f"{final_path}_backup_{datetime.now().strftime('%Y%m%d_%H%M')}"
                    shutil.move(final_path, backup_path)
                    print(f"💾 기존 데이터 백업: {backup_path}")
                
                # 압축 해제된 내용이 training_data 폴더인지 확인
                if 'training_data' in extracted_items:
                    # training_data 폴더가 포함된 경우
                    source_path = os.path.join(temp_extract_path, 'training_data')
                    shutil.move(source_path, final_path)
                elif len(extracted_items) == 1 and os.path.isdir(os.path.join(temp_extract_path, extracted_items[0])):
                    # 단일 폴더인 경우
                    source_path = os.path.join(temp_extract_path, extracted_items[0])
                    shutil.move(source_path, final_path)
                else:
                    # 파일들이 직접 압축된 경우
                    shutil.move(temp_extract_path, final_path)
                
                # 임시 폴더 정리
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                # 압축 해제 결과 확인
                if os.path.exists(final_path):
                    contents = os.listdir(final_path)
                    print(f"📂 최종 경로: {final_path}")
                    print(f"📋 내용: {contents}")
                    
                    # 기본 구조 확인
                    images_dir = os.path.join(final_path, 'images')
                    labels_dir = os.path.join(final_path, 'labels')
                    
                    if os.path.exists(images_dir):
                        image_count = len([f for f in os.listdir(images_dir) 
                                         if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff'))])
                        print(f"📸 이미지: {image_count}개")
                    
                    if os.path.exists(labels_dir):
                        label_count = len([f for f in os.listdir(labels_dir) 
                                         if f.endswith('.txt')])
                        print(f"🏷️ 라벨: {label_count}개")
                
                return final_path
                
            except Exception as e:
                print(f"❌ ZIP 압축 해제 실패: {e}")
                return None
        
        else:
            print("❌ ZIP 파일을 찾을 수 없습니다.")
            print("💡 다음 경로에 ZIP 파일을 업로드하세요:")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data/your_data.zip")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data.zip")
            print("  📦 /content/drive/MyDrive/training_data.zip")
            print("  📦 /content/drive/MyDrive/dataset.zip")
            return None
    
    # ZIP 파일 압축 해제 실행
    extracted_data_path = find_and_extract_training_zip()
    
    # 학습 데이터 경로 확인 (압축 해제 결과 포함)
    data_paths = [
        extracted_data_path,  # 압축 해제된 경로 우선
        '/content/drive/MyDrive/pinetree_scan/training_data',
        '/content/training_data',
        '/content/drive/MyDrive/training_data'
    ]
    
    data_path = None
    for path in data_paths:
        if path and os.path.exists(path):
            # data.yaml이 있거나 images 폴더가 있으면 유효한 경로로 간주
            if (os.path.exists(f"{path}/data.yaml") or 
                os.path.exists(f"{path}/images")):
                data_path = path
                print(f"✅ 학습 데이터 발견: {data_path}")
                break
    
    if not data_path:
        print("⚠️ 학습 데이터를 찾을 수 없습니다.")
        print("📋 필요한 파일 구조:")
        print("  📦 ZIP 파일:")
        print("    📂 training_data.zip")
        print("      📂 images/ (이미지 파일들)")
        print("      📂 labels/ (라벨 파일들)")
        print("      📄 data.yaml (선택사항)")
        print("")
        print("  📁 또는 폴더 구조:")
        print("    📂 training_data/")
        print("      📄 data.yaml")
        print("      📂 images/")
        print("      📂 labels/")
        data_path = '/content/training_data'  # 기본값
    
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경 감지")
    data_path = './data'  # 로컬 기본 경로

# 📦 필수 패키지 설치
packages = [
    "ultralytics",
    "torch", 
    "torchvision",
    "opencv-python",
    "pillow",
    "matplotlib",
    "numpy",
    "pandas",
    "roboflow"  # 데이터셋 관리용 추가
]

print(f"\n📦 필수 패키지 설치:")
for package in packages:
    install_package(package)

# 📚 라이브러리 임포트
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

print(f"\n🖥️ 시스템 정보:")
print(f"  Python: {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
print(f"  CUDA 사용가능: {'✅' if torch.cuda.is_available() else '❌'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU: {gpu_name}")
    print(f"  GPU 메모리: {gpu_memory:.1f}GB")
    
    # GPU 메모리 정리
    torch.cuda.empty_cache()
    print("  🧹 GPU 메모리 정리 완료")
else:
    print("  ⚠️ GPU를 사용할 수 없습니다. 런타임 설정을 확인하세요!")

# 🏆 기존 학습된 모델 자동 탐지 (강화된 버전)
print(f"\n🔍 기존 학습된 모델 탐지:")

def find_best_model():
    """기존 학습된 모델을 찾는 향상된 함수"""
    # 우선순위별 모델 검색 경로
    search_priorities = [
        # 1순위: 최신 학습 결과 (피해목 탐지 전용)
        {
            'priority': 1,
            'description': '최신 피해목 탐지 모델',
            'patterns': [
                'pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/results/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/best.pt'
            ]
        },
        # 2순위: 일반 학습 결과
        {
            'priority': 2,
            'description': '일반 학습 모델',
            'patterns': [
                'runs/detect/train*/weights/best.pt',
                '/content/runs/detect/train*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/runs/detect/train*/weights/best.pt',
            ]
        },
        # 3순위: 수동 저장 모델
        {
            'priority': 3,
            'description': '수동 저장 모델',
            'patterns': [
                '/content/drive/MyDrive/pinetree_scan/models/best.pt',
                '/content/drive/MyDrive/pinetree_scan/weights/best.pt',
                'models/best.pt',
                'weights/best.pt',
                '/content/best.pt'
            ]
        },
        # 4순위: 기본 모델
        {
            'priority': 4,
            'description': '기본 YOLO 모델',
            'patterns': [
                'yolo11s.pt',
                '/content/yolo11s.pt'
            ]
        }
    ]
    
    found_models = []
    
    for priority_group in search_priorities:
        for pattern in priority_group['patterns']:
            if '*' in pattern:
                matches = glob.glob(pattern)
                for match in matches:
                    if os.path.exists(match):
                        found_models.append({
                            'path': match,
                            'priority': priority_group['priority'],
                            'description': priority_group['description'],
                            'size': os.path.getsize(match) / 1024 / 1024,  # MB
                            'modified': os.path.getctime(match)
                        })
            else:
                if os.path.exists(pattern):
                    found_models.append({
                        'path': pattern,
                        'priority': priority_group['priority'],
                        'description': priority_group['description'],
                        'size': os.path.getsize(pattern) / 1024 / 1024,  # MB
                        'modified': os.path.getctime(pattern)
                    })
    
    if found_models:
        # 우선순위와 수정시간으로 정렬
        found_models.sort(key=lambda x: (x['priority'], -x['modified']))
        return found_models
    
    return None

# 모델 검색 실행
found_models = find_best_model()

if found_models:
    print(f"🎯 발견된 모델: {len(found_models)}개")
    
    # 상위 3개 모델 표시
    for i, model_info in enumerate(found_models[:3]):
        emoji = "🏆" if i == 0 else "🥈" if i == 1 else "🥉"
        print(f"  {emoji} {model_info['description']}")
        print(f"     경로: {model_info['path']}")
        print(f"     크기: {model_info['size']:.1f}MB")
        print(f"     수정: {datetime.fromtimestamp(model_info['modified']).strftime('%Y-%m-%d %H:%M')}")
        
        # 첫 번째 모델 선택
        if i == 0:
            selected_model = model_info['path']
    
    # 선택된 모델 로드 및 검증
    print(f"\n📥 선택된 모델 로드: {os.path.basename(selected_model)}")
    
    try:
        temp_model = YOLO(selected_model)
        
        # 모델 정보 확인
        print(f"✅ 모델 로드 성공!")
        print(f"  📊 모델 정보:")
        
        if hasattr(temp_model, 'model') and hasattr(temp_model.model, 'names'):
            class_names = list(temp_model.model.names.values())
            print(f"    🏷️ 클래스 수: {len(class_names)}")
            print(f"    🏷️ 클래스명: {class_names}")
            
            # 소나무/피해목 관련 클래스 확인
            pine_related = any(keyword in ' '.join(class_names).lower() 
                             for keyword in ['pine', 'damage', 'tree', '피해', '소나무', 'damaged'])
            
            if pine_related:
                print(f"    🌲 소나무 피해목 탐지용 모델 확인!")
            else:
                print(f"    ⚠️ 일반 객체 탐지 모델 (전이학습 예정)")
        
        # 모델 아키텍처 정보
        try:
            model_yaml = temp_model.model.yaml
            if 'backbone' in str(model_yaml):
                print(f"    🏗️ 아키텍처: YOLOv11s")
        except:
            print(f"    🏗️ 아키텍처: YOLO 계열")
            
        existing_model = selected_model
        del temp_model  # 메모리 정리
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        print(f"📥 기본 YOLOv11s 모델로 대체")
        existing_model = 'yolo11s.pt'

else:
    print("❌ 기존 학습 모델을 찾을 수 없습니다.")
    print("📥 기본 YOLOv11s 모델 사용")
    existing_model = 'yolo11s.pt'

# 🎯 모델 사용 계획 안내
print(f"\n🎯 모델 사용 계획:")
if 'yolo11s.pt' in existing_model:
    print(f"  📝 기본 YOLOv11s → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1-2시간 (처음부터)")
    print(f"  📈 예상 성능: 85-95% mAP50")
elif 'damage_detection' in existing_model:
    print(f"  📝 기존 피해목 모델 → 추가 학습/미세조정")
    print(f"  🎯 예상 학습 시간: 30분-1시간 (미세조정)")
    print(f"  📈 예상 성능: 90-98% mAP50")
else:
    print(f"  📝 기존 모델 → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1시간 (전이학습)")
    print(f"  📈 예상 성능: 87-95% mAP50")

# 🗂️ 출력 디렉토리 설정
if IN_COLAB:
    output_base = '/content/drive/MyDrive/pinetree_scan/results'
else:
    output_base = './results'

os.makedirs(output_base, exist_ok=True)
print(f"\n📁 결과 저장 경로: {output_base}")

# 📊 데이터셋 정보 확인
data_yaml_path = f"{data_path}/data.yaml"
if os.path.exists(data_yaml_path):
    print(f"\n📊 데이터셋 정보:")
    try:
        with open(data_yaml_path, 'r') as f:
            content = f.read()
            print(f"  📄 data.yaml 내용 확인됨")
            if 'names:' in content:
                print(f"  🏷️ 클래스 정보 포함됨")
    except:
        print(f"  ⚠️ data.yaml 읽기 실패")
else:
    print(f"\n⚠️ 데이터셋 파일 없음: {data_yaml_path}")
    print("📋 data.yaml 파일이 필요합니다!")

# 환경 변수 설정
globals()['existing_model'] = existing_model
globals()['data_path'] = data_path
globals()['output_base'] = output_base
globals()['IN_COLAB'] = IN_COLAB
globals()['found_models'] = found_models

print(f"\n✅ 환경 설정 완료!")
print(f"  🏆 사용할 모델: {os.path.basename(existing_model)}")
print(f"  📂 모델 경로: {existing_model}")
print(f"  📊 데이터 경로: {data_path}")
print(f"  📁 결과 경로: {output_base}")
print("=" * 50)

## 📊 **1.5단계: 학습 데이터 전처리 (바운딩 박스 최적화)**

> 개별 소나무 탐지를 위해 바운딩 박스 크기를 24px로 최적화합니다.  
> 기존 64px에서 24px로 줄여 한 박스당 하나의 나무만 포함되도록 조정합니다.

In [ ]:
# 📊 1.5단계: 학습 데이터 전처리 (바운딩 박스 24px 최적화)
print("📊 학습 데이터 전처리 시작!")
print("🎯 바운딩 박스 크기를 24px로 최적화 (개별 나무 탐지)")
print("=" * 60)

# 환경 변수 확인
if 'data_path' not in globals():
    data_path = '/content/training_data'

# 📋 바운딩 박스 최적화 설정
BBOX_CONFIG = {
    'size': 24,  # 24x24 픽셀 (기존 64px에서 축소)
    'overlap': 0.2,  # 20% 겹침
    'min_tree_size': 12,  # 최소 나무 크기 (픽셀)
    'max_tree_size': 36,  # 최대 나무 크기 (픽셀)
}

print(f"🎯 바운딩 박스 설정:")
print(f"  📏 크기: {BBOX_CONFIG['size']}x{BBOX_CONFIG['size']} 픽셀")
print(f"  🔄 겹침률: {BBOX_CONFIG['overlap']*100}%")
print(f"  🌲 나무 크기 범위: {BBOX_CONFIG['min_tree_size']}-{BBOX_CONFIG['max_tree_size']} 픽셀")

# 🔧 데이터 전처리 함수 정의
def preprocess_training_data(data_path, bbox_size=24):
    """
    학습 데이터 전처리 - 바운딩 박스 크기 최적화
    """
    import yaml
    import shutil
    from pathlib import Path
    
    print(f"📂 데이터 경로 확인: {data_path}")
    
    # 기존 데이터 백업
    if os.path.exists(data_path):
        backup_path = f"{data_path}_backup_{datetime.now().strftime('%Y%m%d_%H%M')}"
        if not os.path.exists(backup_path):
            shutil.copytree(data_path, backup_path)
            print(f"💾 기존 데이터 백업: {backup_path}")
    
    # data.yaml 파일 확인 및 생성
    data_yaml_path = f"{data_path}/data.yaml"
    
    if not os.path.exists(data_yaml_path):
        print("📝 data.yaml 파일 생성 중...")
        
        # 기본 data.yaml 생성
        data_config = {
            'path': data_path,
            'train': f"{data_path}/train/images",
            'val': f"{data_path}/val/images",  # 검증용
            'test': f"{data_path}/val/images",  # 테스트용 (val과 동일)
            'nc': 1,  # 클래스 수 (소나무 피해목)
            'names': ['damaged_pine'],  # 클래스 이름
            
            # 추가 메타데이터
            'bbox_size': bbox_size,
            'created_date': datetime.now().isoformat(),
            'description': 'Pine tree damage detection dataset with optimized 24px bounding boxes'
        }
        
        os.makedirs(data_path, exist_ok=True)
        with open(data_yaml_path, 'w') as f:
            yaml.dump(data_config, f, default_flow_style=False)
        
        print(f"✅ data.yaml 생성 완료: {data_yaml_path}")
    else:
        print(f"✅ 기존 data.yaml 사용: {data_yaml_path}")
        
        # 기존 파일에 bbox_size 설정 추가
        try:
            with open(data_yaml_path, 'r') as f:
                data_config = yaml.safe_load(f)
            
            # 경로 설정 확인 및 수정
            data_config['path'] = data_path
            
            # YOLO 표준 구조 확인 후 경로 설정
            if os.path.exists(f"{data_path}/train/images"):
                print("🔧 YOLO 표준 구조로 경로 설정")
                data_config['train'] = f"{data_path}/train/images"
                data_config['val'] = f"{data_path}/val/images"
                data_config['test'] = f"{data_path}/val/images"
            else:
                print("🔧 단순 구조로 경로 설정")
                data_config['train'] = f"{data_path}/images"
                data_config['val'] = f"{data_path}/images"
                data_config['test'] = f"{data_path}/images"
            
            if 'bbox_size' not in data_config:
                data_config['bbox_size'] = bbox_size
                data_config['updated_date'] = datetime.now().isoformat()
                print(f"🔧 data.yaml에 bbox_size={bbox_size} 설정 추가")
            
            # 업데이트된 설정 저장
            with open(data_yaml_path, 'w') as f:
                yaml.dump(data_config, f, default_flow_style=False)
            print(f"🔧 data.yaml 경로 설정 업데이트")
            
        except Exception as e:
            print(f"⚠️ data.yaml 업데이트 실패: {e}")
    
    # 필요한 디렉토리 생성
    required_dirs = ['images', 'labels']
    for dir_name in required_dirs:
        dir_path = f"{data_path}/{dir_name}"
        os.makedirs(dir_path, exist_ok=True)
        print(f"📁 디렉토리 확인: {dir_path}")
    
    return data_yaml_path

# 🛠️ 바운딩 박스 라벨 최적화 함수
def optimize_bbox_labels(data_path, target_size=24):
    """
    기존 라벨 파일의 바운딩 박스 크기를 최적화
    """
    # YOLO 표준 구조 확인
    train_labels_dir = f"{data_path}/train/labels"
    val_labels_dir = f"{data_path}/val/labels"
    simple_labels_dir = f"{data_path}/labels"
    
    label_dirs = []
    
    # 존재하는 라벨 디렉토리 찾기
    if os.path.exists(train_labels_dir):
        label_dirs.append(train_labels_dir)
    if os.path.exists(val_labels_dir):
        label_dirs.append(val_labels_dir)
    if os.path.exists(simple_labels_dir):
        label_dirs.append(simple_labels_dir)
    
    if not label_dirs:
        print(f"⚠️ 라벨 디렉토리 없음")
        return False
    
    import glob
    total_optimized = 0
    total_files = 0
    
    for labels_dir in label_dirs:
        print(f"🔧 라벨 최적화: {labels_dir}")
        
        label_files = glob.glob(f"{labels_dir}/*.txt")
        
        if not label_files:
            print(f"  ⚠️ 라벨 파일 없음: {labels_dir}")
            continue
        
        print(f"  📋 라벨 파일: {len(label_files)}개")
        total_files += len(label_files)
        
        optimized_count = 0
        
        for label_file in label_files:
            try:
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                
                optimized_lines = []
                modified = False
                
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id, x_center, y_center, width, height = parts[:5]
                        
                        # 현재 크기를 픽셀 단위로 가정 (640x640 기준)
                        current_width = float(width) * 640
                        current_height = float(height) * 640
                        
                        # 크기가 target_size보다 큰 경우 조정
                        if current_width > target_size or current_height > target_size:
                            # target_size로 정규화
                            new_width = min(target_size / 640, float(width))
                            new_height = min(target_size / 640, float(height))
                            
                            optimized_lines.append(f"{class_id} {x_center} {y_center} {new_width:.6f} {new_height:.6f}\n")
                            modified = True
                        else:
                            optimized_lines.append(line)
                    else:
                        optimized_lines.append(line)
                
                if modified:
                    with open(label_file, 'w') as f:
                        f.writelines(optimized_lines)
                    optimized_count += 1
                    
            except Exception as e:
                print(f"    ⚠️ 라벨 파일 처리 실패 {label_file}: {e}")
        
        print(f"  ✅ 최적화 완료: {optimized_count}/{len(label_files)}개")
        total_optimized += optimized_count
    
    print(f"🎯 전체 라벨 최적화 결과: {total_optimized}/{total_files}개 파일 수정됨")
    return True

# 📊 실제 전처리 실행
try:
    # Step 1: 데이터 구조 준비
    processed_data_yaml = preprocess_training_data(data_path, BBOX_CONFIG['size'])
    
    # Step 2: 기존 라벨 최적화 (있는 경우)
    optimize_bbox_labels(data_path, BBOX_CONFIG['size'])
    
    # Step 3: 데이터셋 통계 확인
    images_dir = f"{data_path}/images"
    labels_dir = f"{data_path}/labels"
    train_images_dir = f"{data_path}/train/images"
    train_labels_dir = f"{data_path}/train/labels"
    val_images_dir = f"{data_path}/val/images"
    val_labels_dir = f"{data_path}/val/labels"
    
    image_count = 0
    label_count = 0
    
    # YOLO 표준 구조 확인 (train/val 폴더)
    if os.path.exists(train_images_dir) or os.path.exists(val_images_dir):
        print("📁 YOLO 표준 구조 감지됨 (train/val)")
        
        # train 폴더 확인
        if os.path.exists(train_images_dir):
            import glob
            for ext in ['jpg', 'jpeg', 'png', 'tif', 'tiff']:
                pattern = f"{train_images_dir}/*.{ext}"
                files = glob.glob(pattern)
                image_count += len(files)
        
        if os.path.exists(train_labels_dir):
            label_files = glob.glob(f"{train_labels_dir}/*.txt")
            label_count += len(label_files)
        
        # val 폴더 확인
        if os.path.exists(val_images_dir):
            for ext in ['jpg', 'jpeg', 'png', 'tif', 'tiff']:
                pattern = f"{val_images_dir}/*.{ext}"
                files = glob.glob(pattern)
                image_count += len(files)
        
        if os.path.exists(val_labels_dir):
            label_files = glob.glob(f"{val_labels_dir}/*.txt")
            label_count += len(label_files)
            
        print(f"📸 이미지 파일: {image_count}개 (train + val)")
        print(f"🏷️ 라벨 파일: {label_count}개 (train + val)")
        
    # 단순 구조 확인 (images/labels 폴더)
    elif os.path.exists(images_dir):
        print("📁 단순 구조 감지됨 (images/labels)")
        
        import glob
        for ext in ['jpg', 'jpeg', 'png', 'tif', 'tiff']:
            pattern = f"{images_dir}/*.{ext}"
            files = glob.glob(pattern)
            image_count += len(files)
            
        print(f"📸 이미지 파일: {image_count}개")
    else:
        print(f"📸 이미지 파일: 없음 (추후 업로드 필요)")
    
    if os.path.exists(labels_dir):
        label_files = glob.glob(f"{labels_dir}/*.txt")
        label_count = len(label_files)
        print(f"🏷️ 라벨 파일: {label_count}개")
    else:
        print(f"🏷️ 라벨 파일: 없음 (추후 생성 필요)")
    
    # Step 4: 최종 설정 확인
    print(f"\n📋 전처리 완료 요약:")
    print(f"  📂 데이터 경로: {data_path}")
    print(f"  📄 설정 파일: {processed_data_yaml}")
    print(f"  📏 바운딩 박스: {BBOX_CONFIG['size']}px (최적화됨)")
    print(f"  🎯 개별 나무 탐지: 활성화")
    
    # 전역 변수 업데이트
    globals()['data_path'] = data_path
    globals()['processed_data_yaml'] = processed_data_yaml
    globals()['bbox_optimized'] = True
    
    preprocessing_completed = True
    
except Exception as e:
    print(f"❌ 전처리 실패: {e}")
    print(f"\n🔧 문제 해결 방법:")
    print(f"  1. Google Drive에 training_data 폴더가 있는지 확인")
    print(f"  2. images/ 및 labels/ 폴더 구조 확인")
    print(f"  3. 파일 권한 확인")
    print(f"  4. 경로 설정 재확인")
    
    preprocessing_completed = False

# 💡 사용자 안내
print(f"\n💡 다음 단계 안내:")

if preprocessing_completed:
    print(f"  ✅ 전처리 완료! 다음 셀(2단계)로 진행하세요.")
    print(f"  🎯 24px 바운딩 박스로 개별 나무 탐지 최적화됨")
else:
    print(f"  ⚠️ 전처리 실패. 데이터를 확인하고 재시도하세요.")
    print(f"  📋 필요한 데이터 구조:")
    print(f"    📂 training_data/")
    print(f"      📄 data.yaml")
    print(f"      📂 images/ (이미지 파일들)")
    print(f"      📂 labels/ (라벨 파일들)")

print(f"\n📈 바운딩 박스 최적화 효과:")
print(f"  🎯 기존 64px → 24px: 개별 나무 탐지 향상")
print(f"  🔍 오탐지 감소: 한 박스당 하나의 나무")
print(f"  📊 정확도 향상: 더 정밀한 위치 탐지")
print(f"  ⚡ 학습 효율성: 최적화된 바운딩 박스 크기")

globals()['preprocessing_completed'] = preprocessing_completed

print(f"\n✅ 데이터 전처리 단계 완료!")
print("=" * 60)

## 🚀 **2단계: YOLOv11s 소나무 전용 학습**

> 기존 모델을 활용한 전이학습으로 소나무 피해목 탐지 성능을 극대화합니다.  
> **네이밍**: `damage_detection_YYYYMMDD_HHMM` 형식으로 자동 생성

In [ ]:
# 🚀 2단계: YOLOv11s 소나무 전용 학습 실행
print("🌲 YOLOv11s 소나무 피해목 탐지 학습 시작!")
print("🎯 24px 바운딩 박스 최적화 + 전이학습으로 성능 극대화!")
print("=" * 60)

# 환경 변수 및 전처리 결과 확인
if 'existing_model' not in globals():
    print("⚠️ 환경 설정이 완료되지 않았습니다. 1단계를 먼저 실행하세요!")
    existing_model = 'yolo11s.pt'

if 'preprocessing_completed' not in globals():
    print("⚠️ 데이터 전처리가 완료되지 않았습니다. 1.5단계를 먼저 실행하세요!")
    data_path = '/content/training_data'
    bbox_optimized = False
else:
    bbox_optimized = globals().get('bbox_optimized', False)

if 'processed_data_yaml' in globals():
    data_yaml_path = processed_data_yaml
else:
    data_yaml_path = f"{globals().get('data_path', '/content/training_data')}/data.yaml"

# 📊 데이터셋 최종 확인
print(f"📊 데이터셋 확인: {data_yaml_path}")
print(f"🎯 바운딩 박스 최적화: {'✅ 24px 적용됨' if bbox_optimized else '⚠️ 기본 설정'}")

if not os.path.exists(data_yaml_path):
    print("❌ data.yaml 파일을 찾을 수 없습니다!")
    print("📋 다음 경로들을 확인했습니다:")
    search_paths = [
        '/content/training_data/data.yaml',
        '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml',
        '/content/drive/MyDrive/training_data/data.yaml',
        './data/data.yaml'
    ]
    
    for path in search_paths:
        exists = "✅" if os.path.exists(path) else "❌"
        print(f"  {exists} {path}")
    
    print("\n🔧 해결 방법:")
    print("  1. 1.5단계 데이터 전처리를 먼저 실행하세요")
    print("  2. Google Drive에 training_data 폴더 업로드")
    print("  3. data.yaml, images/, labels/ 폴더 포함 확인")
    
    # 기본값으로 설정하되 경고 표시
    data_yaml_path = '/content/training_data/data.yaml'
    print(f"⚠️ 기본 경로로 설정: {data_yaml_path}")

# 모델 로드
try:
    print(f"📥 모델 로드: {existing_model}")
    model = YOLO(existing_model)
    print(f"✅ 모델 로드 성공!")
    
    # 모델 정보 출력
    try:
        if hasattr(model, 'model') and hasattr(model.model, 'names'):
            print(f"🏷️ 클래스 수: {len(model.model.names)}")
            print(f"🏷️ 클래스: {list(model.model.names.values())}")
    except:
        print("📝 모델 정보 확인 중...")
    
    model_ready = True
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("🔧 기본 YOLOv11s 모델로 재시도...")
    
    try:
        model = YOLO('yolo11s.pt')
        print("✅ 기본 모델 로드 성공!")
        existing_model = 'yolo11s.pt'
        model_ready = True
    except Exception as e2:
        print(f"❌ 기본 모델 로드도 실패: {e2}")
        model_ready = False

if model_ready:
    # GPU 설정
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🖥️ 사용 장치: {device.upper()}")
    
    # GPU 메모리 정리
    if device == 'cuda':
        torch.cuda.empty_cache()
        available_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"💾 사용 가능 GPU 메모리: {available_memory:.1f}GB")
        
        # 메모리에 따른 배치 크기 자동 조정
        if available_memory >= 15:
            batch_size = 16
        elif available_memory >= 8:
            batch_size = 8
        else:
            batch_size = 4
    else:
        batch_size = 2
        print("⚠️ CPU 모드: 학습 속도가 매우 느려집니다!")
    
    # 🌲 소나무 전용 최적화 학습 설정 (24px 바운딩 박스 특화)
    training_config = {
        # 기본 설정
        'data': data_yaml_path,  # 동적 경로 사용
        'epochs': 200,
        'batch': batch_size,  # 메모리에 따른 자동 조정
        'imgsz': 640,
        'patience': 100,
        'save': True,
        'save_period': 10,
        'cache': 'ram' if device == 'cuda' else False,  # GPU일 때만 RAM 캐시
        'device': device,
        'workers': 4 if device == 'cuda' else 2,
        
        # 🎯 요청된 네이밍 규칙 (24px 최적화 표시)
        'project': 'pinetree_yolov11s',  # 프로젝트명 유지
        'name': f'damage_detection_24px_{datetime.now().strftime("%Y%m%d_%H%M")}',
        
        # 24px 바운딩 박스 특화 전이학습 최적화
        'optimizer': 'AdamW',
        'lr0': 0.0002,              # 작은 바운딩 박스용 낮은 학습률
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0003,     # 과적합 방지 강화
        'warmup_epochs': 20,        # 안정적 학습을 위한 긴 워밍업
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.005,
        
        # 24px 소나무 특화 손실 함수 (개별 나무 탐지 최적화)
        'box': 8.0,                 # 바운딩 박스 정확도 강화
        'cls': 0.4,                 # 분류 손실 적당히
        'dfl': 2.0,                 # 분포 초점 손실 강화 (정밀한 위치)
        
        # 24px 최적화 데이터 증강 (작은 객체용)
        'hsv_h': 0.003,             # 소나무 고유색 더 강하게 보존
        'hsv_s': 0.2,               # 채도 변화 최소화
        'hsv_v': 0.15,              # 밝기 변화 최소화
        'degrees': 0.0,             # 회전 비활성화 (작은 객체)
        'translate': 0.03,          # 이동 최소화
        'scale': 0.1,               # 스케일 변화 최소화 (24px 유지)
        'shear': 0.0,               # 전단 비활성화
        'perspective': 0.0,         # 원근 비활성화
        'flipud': 0.0,              # 상하 뒤집기 비활성화
        'fliplr': 0.2,              # 좌우 뒤집기만 약간
        'mosaic': 0.3,              # 모자이크 줄임 (작은 객체 보호)
        'mixup': 0.0,               # 믹스업 비활성화
        'copy_paste': 0.0,          # 복사-붙여넣기 비활성화
        
        # 24px 안정성 설정
        'amp': True if device == 'cuda' else False,
        'fraction': 1.0,
        'profile': False,
        'freeze': None,
        'dropout': 0.15,            # 과적합 방지 강화
        'val': True,
        'plots': True,
        'save_json': True,
        'verbose': True,
        'seed': 42,
        
        # 24px 소나무 전용 설정
        'rect': False,
        'single_cls': True,         # 단일 클래스 (소나무 피해목)
        'overlap_mask': True,
        'mask_ratio': 2,            # 작은 객체용 마스크 비율
        'cos_lr': True,
        'close_mosaic': 75,         # 모자이크 일찍 종료 (작은 객체 보호)
        
        # 24px 오탐지 방지 임계값
        'conf': 0.35,               # 24px용 적당한 신뢰도
        'iou': 0.5,                 # 작은 객체용 IoU
    }
    
    # 실행명 및 설정 출력
    run_name = f'damage_detection_24px_{datetime.now().strftime("%Y%m%d_%H%M")}'
    print(f"\n🎯 24px 최적화 학습 설정:")
    print(f"  📂 프로젝트: pinetree_yolov11s")
    print(f"  📅 실행명: {run_name}")
    print(f"  🏆 기존 모델: {existing_model}")
    print(f"  📊 데이터: {data_yaml_path}")
    print(f"  📏 바운딩 박스: 24px (개별 나무 최적화)")
    print(f"  🎯 배치 크기: {batch_size} (메모리 자동 조정)")
    print(f"  🌲 전용 최적화: 24px 소나무 특화 + 오탐지 방지")
    print(f"  💾 캐시: {'RAM' if training_config['cache'] == 'ram' else 'Disk'}")
    
    print(f"\n🔥 24px 바운딩 박스 최적화 학습 시작!")
    print(f"⏰ 예상 시간: 1-2시간 (GPU 성능에 따라 변동)")
    print(f"🎯 개별 나무 탐지 정확도 극대화!")
    
    try:
        # 학습 실행
        results = model.train(**training_config)
        
        print(f"\n🎉 24px 최적화 학습 완료!")
        print(f"📈 최종 mAP50: {results.box.map50:.4f}")
        print(f"📈 최종 mAP50-95: {results.box.map:.4f}")
        print(f"📈 Precision: {results.box.mp:.4f}")
        print(f"📈 Recall: {results.box.mr:.4f}")
        
        # 24px 최적화 성능 평가
        if results.box.mp > 0.85:
            print("🌟 탁월한 정밀도! 24px 최적화로 오탐지 대폭 감소!")
        elif results.box.mp > 0.7:
            print("✅ 우수한 정밀도! 개별 나무 탐지 성공!")
        
        if results.box.mr > 0.7:
            print("🌟 뛰어난 재현율! 작은 피해목도 놓치지 않음!")
        elif results.box.mr > 0.5:
            print("✅ 좋은 재현율! 피해목 탐지 성공!")
        
        if results.box.map50 > 0.9:
            print("🏆 최고 등급 mAP50! 24px 최적화 대성공!")
        elif results.box.map50 > 0.8:
            print("🌟 우수한 mAP50! 실용적 성능 달성!")
        
        # 결과를 Google Drive에 백업 (Colab 환경에서)
        if 'IN_COLAB' in globals() and IN_COLAB:
            try:
                result_source = f"pinetree_yolov11s/{run_name}"
                result_backup = f"/content/drive/MyDrive/pinetree_scan/results/{run_name}"
                
                if os.path.exists(result_source):
                    import shutil
                    shutil.copytree(result_source, result_backup, dirs_exist_ok=True)
                    print(f"💾 24px 최적화 결과를 Google Drive에 백업: {result_backup}")
            except Exception as e:
                print(f"⚠️ Drive 백업 실패: {e}")
        
        training_completed = True
        training_results = results
        
    except Exception as e:
        print(f"❌ 학습 오류: {e}")
        print(f"\n🔧 문제 해결 방법:")
        print(f"  1. GPU 메모리 부족: 배치 크기를 더 줄이기 (batch=2)")
        print(f"  2. 데이터 경로 오류: 1.5단계 전처리 재실행")
        print(f"  3. 24px 데이터 문제: data.yaml 파일 확인")
        print(f"  4. 권한 문제: Google Drive 마운트 재시도")
        print(f"  5. 런타임 재시작 후 재시도")
        
        training_completed = False
        training_results = None
        
else:
    print("❌ 모델 준비 실패")
    print("🔧 문제 해결:")
    print("  1. 인터넷 연결 확인")
    print("  2. ultralytics 패키지 재설치")
    print("  3. 런타임 재시작")
    training_completed = False
    training_results = None

# 전역 변수 설정
globals()['training_completed'] = training_completed
globals()['training_results'] = training_results
if 'run_name' in locals():
    globals()['run_name'] = run_name

print(f"\n📋 24px 최적화 학습 상태: training_completed = {training_completed}")
print(f"🎯 개별 나무 탐지: {'✅ 최적화됨' if bbox_optimized else '⚠️ 기본 설정'}")
print("=" * 60)

## 📊 **3단계: 결과 분석 및 시각화**

> 학습 결과를 자동으로 분석하고 성능 지표, 학습 곡선, 예측 결과를 시각화합니다.

In [ ]:
# ? 3단계: 학습된 모델 성능 검증
print("? 학습 완료 모델 성능 검증!")
print("📊 학습 결과 분석 및 추론 테스트")
print("=" * 60)

# 이전 단계 확인
if not globals().get('training_completed', False):
    print("⚠️ 학습이 완료되지 않았습니다!")
    print("💡 추론만 테스트하려면 아래 코드를 실행하세요:")
    
    # 최신 모델 자동 탐지
    best_model_path = None
    search_paths = [
        # 현재 디렉토리
        './pinetree_yolov11s/damage_detection_*/weights/best.pt',
        # Colab 기본 경로
        '/content/pinetree_yolov11s/damage_detection_*/weights/best.pt',
        # Drive 백업 경로
        '/content/drive/MyDrive/pinetree_scan/results/damage_detection_*/weights/best.pt',
        # 기존 results 폴더
        '/content/drive/MyDrive/pinetree_scan/results/*/weights/best.pt',
    ]
    
    import glob
    for pattern in search_paths:
        matches = glob.glob(pattern)
        if matches:
            # 가장 최신 모델 선택
            best_model_path = max(matches, key=os.path.getctime)
            break
    
    if best_model_path:
        print(f"🎯 최신 학습 모델 발견: {best_model_path}")
        model = YOLO(best_model_path)
        training_completed = True
    else:
        print("❌ 학습된 모델을 찾을 수 없습니다. YOLOv11s 기본 모델 사용")
        model = YOLO('yolo11s.pt')
        best_model_path = 'yolo11s.pt'

else:
    # 학습 완료된 경우 최고 성능 모델 로드
    if 'run_name' in globals():
        best_model_path = f"pinetree_yolov11s/{run_name}/weights/best.pt"
    else:
        # 최신 모델 자동 탐지
        import glob
        matches = glob.glob('./pinetree_yolov11s/damage_detection_*/weights/best.pt')
        if matches:
            best_model_path = max(matches, key=os.path.getctime)
        else:
            best_model_path = 'yolo11s.pt'
    
    try:
        model = YOLO(best_model_path)
        print(f"✅ 학습 완료 모델 로드: {best_model_path}")
    except Exception as e:
        print(f"⚠️ 모델 로드 실패: {e}")
        model = YOLO('yolo11s.pt')
        best_model_path = 'yolo11s.pt'

# 📊 학습 결과 분석
if training_completed and 'training_results' in globals() and training_results:
    print(f"\n📈 학습 결과 요약:")
    results = training_results
    
    try:
        print(f"  🎯 최종 mAP50: {results.box.map50:.4f} ({results.box.map50*100:.1f}%)")
        print(f"  🎯 최종 mAP50-95: {results.box.map:.4f} ({results.box.map*100:.1f}%)")
        print(f"  🎯 Precision: {results.box.mp:.4f} ({results.box.mp*100:.1f}%)")
        print(f"  🎯 Recall: {results.box.mr:.4f} ({results.box.mr*100:.1f}%)")
        
        # 성능 평가
        performance_score = (results.box.map50 + results.box.mp + results.box.mr) / 3
        
        if performance_score > 0.85:
            print("🌟 탁월! 프로덕션 레벨 성능입니다!")
        elif performance_score > 0.7:
            print("✅ 우수! 실용적인 성능입니다!")
        elif performance_score > 0.5:
            print("⚠️ 보통. 추가 학습이 필요할 수 있습니다.")
        else:
            print("❌ 낮음. 데이터나 설정을 재검토하세요.")
            
    except Exception as e:
        print(f"📊 결과 분석 중 오류: {e}")

# 🖼️ 추론 테스트용 이미지 준비
print(f"\n🖼️ 테스트 이미지 검색...")

# 테스트 이미지 경로 후보들
test_image_paths = []
image_search_paths = [
    # 환경 설정에서 확인된 data_path 사용
    f"{globals().get('data_path', '/content/training_data')}/images/*.{ext}"
    for ext in ['jpg', 'jpeg', 'png', 'tif', 'tiff']
] + [
    # 추가 검색 경로들
    '/content/drive/MyDrive/pinetree_scan/data/images/*.jpg',
    '/content/drive/MyDrive/pinetree_scan/data/images/*.tif',
    '/content/drive/MyDrive/training_data/images/*.jpg',
    '/content/drive/MyDrive/training_data/images/*.tif',
    './data/images/*.jpg',
    './data/images/*.tif',
]

import glob
for pattern in image_search_paths:
    matches = glob.glob(pattern)
    test_image_paths.extend(matches[:3])  # 각 패턴에서 최대 3개

# 중복 제거
test_image_paths = list(set(test_image_paths))[:5]  # 최대 5개

if test_image_paths:
    print(f"✅ 테스트 이미지 {len(test_image_paths)}개 발견!")
    for i, path in enumerate(test_image_paths):
        print(f"  {i+1}. {os.path.basename(path)}")
else:
    print("❌ 테스트 이미지를 찾을 수 없습니다!")
    print("💡 Google Drive에 이미지를 업로드하거나 URL로 테스트하세요.")
    
    # 샘플 URL 이미지 (drone/aerial view)
    test_image_paths = [
        "https://images.unsplash.com/photo-1502780402662-acc01917610e?w=800",  # forest aerial
        "https://images.unsplash.com/photo-1441974231531-c6227db76b6e?w=800"   # forest damage
    ]
    print("🌐 인터넷 샘플 이미지로 테스트합니다.")

# 🔍 추론 실행 및 시각화
print(f"\n🔍 추론 테스트 시작!")

inference_results = []
successful_tests = 0

for i, image_path in enumerate(test_image_paths[:3]):  # 최대 3개만 테스트
    try:
        print(f"\n📸 이미지 {i+1}: {os.path.basename(image_path)}")
        
        # 추론 실행 (소나무 최적화 설정)
        results = model.predict(
            source=image_path,
            conf=0.3,      # 높은 신뢰도
            iou=0.6,       # 높은 IoU
            save=True,
            save_txt=True,
            save_conf=True,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            verbose=False
        )
        
        # 결과 분석
        detections = results[0]
        num_detections = len(detections.boxes) if detections.boxes is not None else 0
        
        if num_detections > 0:
            confidences = detections.boxes.conf.cpu().numpy()
            avg_conf = np.mean(confidences)
            max_conf = np.max(confidences)
            
            print(f"  🎯 탐지 수: {num_detections}개")
            print(f"  📊 평균 신뢰도: {avg_conf:.3f}")
            print(f"  📊 최고 신뢰도: {max_conf:.3f}")
            
            # 신뢰도 평가
            if max_conf > 0.8:
                print("  🌟 높은 신뢰도 탐지!")
            elif max_conf > 0.5:
                print("  ✅ 적절한 신뢰도")
            else:
                print("  ⚠️ 낮은 신뢰도 - 확인 필요")
                
        else:
            print(f"  ❌ 탐지된 객체 없음")
        
        inference_results.append({
            'image': os.path.basename(image_path),
            'detections': num_detections,
            'avg_confidence': avg_conf if num_detections > 0 else 0,
            'max_confidence': max_conf if num_detections > 0 else 0
        })
        
        successful_tests += 1
        
    except Exception as e:
        print(f"  ❌ 추론 실패: {e}")
        inference_results.append({
            'image': os.path.basename(image_path),
            'error': str(e)
        })

# 📈 추론 결과 요약
print(f"\n📈 추론 테스트 결과:")
print(f"  ✅ 성공: {successful_tests}/{len(test_image_paths[:3])}")

if successful_tests > 0:
    total_detections = sum(r.get('detections', 0) for r in inference_results)
    avg_confidence = np.mean([r.get('avg_confidence', 0) for r in inference_results if r.get('detections', 0) > 0])
    
    print(f"  🎯 총 탐지: {total_detections}개")
    print(f"  📊 전체 평균 신뢰도: {avg_confidence:.3f}")
    
    # 상세 결과
    for result in inference_results:
        if 'error' not in result:
            print(f"    📸 {result['image']}: {result['detections']}개 (신뢰도: {result['max_confidence']:.3f})")

# 🎯 추론 품질 평가
if successful_tests > 0:
    print(f"\n🎯 모델 품질 평가:")
    
    # 탐지율 평가
    detection_rate = sum(1 for r in inference_results if r.get('detections', 0) > 0) / successful_tests
    print(f"  📊 탐지율: {detection_rate*100:.1f}%")
    
    if detection_rate > 0.8:
        print("  🌟 탁월한 탐지 성능!")
    elif detection_rate > 0.5:
        print("  ✅ 적절한 탐지 성능")
    else:
        print("  ⚠️ 탐지 성능 개선 필요")
    
    # 신뢰도 평가
    if successful_tests > 0:
        high_conf_rate = sum(1 for r in inference_results if r.get('max_confidence', 0) > 0.7) / successful_tests
        print(f"  ? 고신뢰도 탐지율: {high_conf_rate*100:.1f}%")
        
        if high_conf_rate > 0.7:
            print("  🌟 높은 신뢰도! 오탐지 위험 낮음")
        elif high_conf_rate > 0.4:
            print("  ✅ 적절한 신뢰도")
        else:
            print("  ⚠️ 신뢰도 개선 필요")

# 결과 저장
if 'IN_COLAB' in globals() and IN_COLAB and successful_tests > 0:
    try:
        results_dir = f"/content/drive/MyDrive/pinetree_scan/inference_results/{datetime.now().strftime('%Y%m%d_%H%M')}"
        os.makedirs(results_dir, exist_ok=True)
        
        # 추론 결과를 JSON으로 저장
        import json
        with open(f"{results_dir}/inference_summary.json", 'w') as f:
            json.dump({
                'model_path': best_model_path,
                'test_date': datetime.now().isoformat(),
                'successful_tests': successful_tests,
                'total_detections': total_detections,
                'avg_confidence': float(avg_confidence) if successful_tests > 0 else 0,
                'detection_rate': detection_rate,
                'results': inference_results
            }, f, indent=2)
        
        print(f"\n? 추론 결과 저장: {results_dir}")
        
    except Exception as e:
        print(f"⚠️ 결과 저장 실패: {e}")

print(f"\n📋 추론 테스트 완료!")
print("=" * 60)

## 🏆 **4단계: 모델 성능 평가**

> 최종 성능 지표 요약 및 모델 품질 평가를 수행합니다.

In [ ]:
# ? 4단계: 최종 결과 정리 및 실용화 가이드
print("? YOLOv11s 소나무 피해목 탐지 시스템 완성!")
print("📋 최종 결과 요약 및 실용화 방법 안내")
print("=" * 60)

# 📊 전체 결과 종합
completion_status = {
    'environment_setup': 'environment_configured' in globals(),
    'training_completed': globals().get('training_completed', False),
    'inference_tested': 'inference_results' in globals() or 'successful_tests' in globals(),
    'model_ready': 'model' in globals()
}

print("📊 시스템 완성도 체크:")
for step, status in completion_status.items():
    emoji = "✅" if status else "❌"
    step_name = {
        'environment_setup': '환경 설정',
        'training_completed': '모델 학습',
        'inference_tested': '추론 테스트',
        'model_ready': '모델 준비'
    }[step]
    print(f"  {emoji} {step_name}: {'완료' if status else '미완료'}")

total_completion = sum(completion_status.values()) / len(completion_status) * 100
print(f"\n🎯 전체 완성도: {total_completion:.0f}%")

# 🏆 성능 결과 요약 (학습이 완료된 경우)
if globals().get('training_completed', False) and 'training_results' in globals():
    print(f"\n🏆 최종 모델 성능:")
    
    try:
        results = training_results
        print(f"  📈 mAP50: {results.box.map50:.4f} ({results.box.map50*100:.1f}%)")
        print(f"  📈 mAP50-95: {results.box.map:.4f} ({results.box.map*100:.1f}%)")
        print(f"  📈 Precision: {results.box.mp:.4f} ({results.box.mp*100:.1f}%)")
        print(f"  📈 Recall: {results.box.mr:.4f} ({results.box.mr*100:.1f}%)")
        
        # 성능 등급 평가
        performance_score = (results.box.map50 + results.box.mp + results.box.mr) / 3
        
        if performance_score > 0.9:
            grade = "A+"
            description = "최고급 - 상용 수준"
        elif performance_score > 0.8:
            grade = "A"
            description = "우수 - 실용적"
        elif performance_score > 0.7:
            grade = "B"
            description = "양호 - 실험적"
        elif performance_score > 0.6:
            grade = "C"
            description = "보통 - 개선 필요"
        else:
            grade = "D"
            description = "미흡 - 재학습 권장"
        
        print(f"  ? 성능 등급: {grade} ({description})")
        
        if 'run_name' in globals():
            print(f"  📂 모델 경로: pinetree_yolov11s/{run_name}/weights/best.pt")
            
    except Exception as e:
        print(f"  ⚠️ 성능 분석 오류: {e}")

# 🔍 추론 결과 요약 (테스트가 완료된 경우)
if 'inference_results' in globals() or 'successful_tests' in globals():
    print(f"\n🔍 추론 테스트 결과:")
    
    try:
        if 'successful_tests' in globals():
            print(f"  ✅ 성공률: {successful_tests}/3 테스트")
        
        if 'total_detections' in globals():
            print(f"  🎯 총 탐지: {total_detections}개")
        
        if 'avg_confidence' in globals() and avg_confidence > 0:
            print(f"  📊 평균 신뢰도: {avg_confidence:.3f}")
            
            if avg_confidence > 0.8:
                print(f"  🌟 높은 신뢰도! 오탐지 위험 낮음")
            elif avg_confidence > 0.6:
                print(f"  ✅ 적절한 신뢰도")
            else:
                print(f"  ⚠️ 신뢰도 개선 권장")
                
    except Exception as e:
        print(f"  ⚠️ 추론 결과 분석 오류: {e}")

# 📁 생성된 파일들 위치 안내
print(f"\n📁 생성된 파일들:")

# 학습 결과 파일들
if 'run_name' in globals():
    model_dir = f"pinetree_yolov11s/{run_name}"
    print(f"  🤖 학습된 모델:")
    print(f"    - 최고 성능: {model_dir}/weights/best.pt")
    print(f"    - 최종 체크포인트: {model_dir}/weights/last.pt")
    print(f"    - 학습 로그: {model_dir}/results.csv")
    print(f"    - 설정 파일: {model_dir}/args.yaml")

# Google Drive 백업 위치
if 'IN_COLAB' in globals() and IN_COLAB:
    print(f"  ☁️ Google Drive 백업:")
    print(f"    - 모델: /content/drive/MyDrive/pinetree_scan/results/")
    print(f"    - 추론 결과: /content/drive/MyDrive/pinetree_scan/inference_results/")

# 🚀 실용화 가이드
print(f"\n🚀 실용화 방법:")

print(f"\n1️⃣ 새로운 이미지 추론:")
if 'best_model_path' in globals():
    print(f"```python")
    print(f"from ultralytics import YOLO")
    print(f"model = YOLO('{best_model_path}')")
    print(f"results = model.predict('your_image.jpg', conf=0.3, save=True)")
    print(f"```")
else:
    print(f"```python")
    print(f"from ultralytics import YOLO")
    print(f"model = YOLO('pinetree_yolov11s/damage_detection_YYYYMMDD_HHMM/weights/best.pt')")
    print(f"results = model.predict('your_image.jpg', conf=0.3, save=True)")
    print(f"```")

print(f"\n2️⃣ 대량 이미지 처리:")
print(f"```python")
print(f"import glob")
print(f"image_paths = glob.glob('your_folder/*.jpg')")
print(f"results = model.predict(image_paths, conf=0.3, save=True)")
print(f"```")

print(f"\n3️⃣ API 서버 구축:")
print(f"```python")
print(f"# FastAPI 서버로 웹 서비스 구축")
print(f"# 기존 API 코드를 참조하여 구현")
print(f"# conf=0.3, iou=0.6 설정 권장")
print(f"```")

print(f"\n4️⃣ 배치 추론 스크립트:")
print(f"```python")
print(f"# 대용량 데이터 처리용")
print(f"for image_batch in image_batches:")
print(f"    results = model.predict(image_batch, conf=0.3)")
print(f"    # 결과 저장 로직")
print(f"```")

# 🔧 최적화 팁
print(f"\n🔧 성능 최적화 팁:")
print(f"  💡 신뢰도 임계값: conf=0.3 (높은 정밀도)")
print(f"  💡 IoU 임계값: iou=0.6 (중복 탐지 방지)")
print(f"  💡 이미지 크기: 640x640 (학습 시와 동일)")
print(f"  💡 GPU 사용: device='cuda' (속도 향상)")
print(f"  💡 배치 처리: 여러 이미지 동시 처리")

# ⚠️ 주의사항 및 권장사항
print(f"\n⚠️ 사용 시 주의사항:")
print(f"  🔸 항공/드론 촬영 이미지에 최적화됨")
print(f"  🔸 소나무 숲 환경에 특화됨")
print(f"  🔸 조명 조건: 낮 시간대 권장")
print(f"  🔸 해상도: 고해상도 이미지 권장")
print(f"  🔸 시즌: 모든 계절 가능 (학습 데이터에 따라)")

# 📞 문제 해결 가이드
print(f"\n?️ 문제 해결:")
print(f"  ❓ 탐지 안됨: conf 값을 0.1~0.2로 낮춤")
print(f"  ❓ 오탐지 많음: conf 값을 0.4~0.5로 높임")
print(f"  ❓ 속도 느림: 이미지 크기 축소 (480x480)")
print(f"  ❓ 메모리 부족: 배치 크기 줄임")
print(f"  ❓ GPU 오류: device='cpu'로 변경")

# 📈 성능 향상 방법
print(f"\n📈 추가 성능 향상:")
print(f"  🎯 더 많은 학습 데이터 추가")
print(f"  🎯 Hard negative mining 적용")
print(f"  🎯 Test-Time Augmentation (TTA)")
print(f"  🎯 모델 앙상블 (여러 모델 조합)")
print(f"  🎯 Post-processing 최적화")

# 🎯 최종 권장사항
print(f"\n🎯 최종 권장사항:")

if total_completion >= 80:
    print(f"  🌟 완성도 높음! 실제 업무에 적용 가능")
    print(f"  🚀 프로덕션 환경 구축 권장")
    print(f"  📊 성능 모니터링 시스템 구축")
elif total_completion >= 60:
    print(f"  ✅ 기본 기능 완성! 추가 테스트 후 적용")
    print(f"  🔧 성능 개선 작업 권장")
    print(f"  📝 더 많은 테스트 데이터 확보")
else:
    print(f"  ⚠️ 추가 작업 필요")
    print(f"  🔄 미완료 단계 재실행")
    print(f"  📋 환경 설정 재점검")

# 📞 지원 정보
print(f"\n📞 추가 지원:")
print(f"  📚 YOLOv11 공식 문서: https://docs.ultralytics.com/")
print(f"  🐙 GitHub 저장소: https://github.com/ultralytics/ultralytics")
print(f"  💬 커뮤니티: https://community.ultralytics.com/")

print(f"\n" + "=" * 60)
print(f"🎉 YOLOv11s 소나무 피해목 탐지 시스템 구축 완료!")
print(f"🌲 지속 가능한 산림 관리에 기여하세요!")
print(f"=" * 60)